In [1]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Initialize the pre-trained MobileNet model without the top classification layer
model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Function to extract features from an image
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Preprocess for MobileNet
    features = model.predict(img_array)
    return features.flatten()

# Directory paths
french_fries_train_dir = '../data/food11/train/french_fries'
french_fries_test_dir = '../data/food11/test/french_fries'

# Collect all image paths in the pizza folders
image_paths = [os.path.join(french_fries_train_dir, fname) for fname in os.listdir(french_fries_train_dir)]
image_paths += [os.path.join(french_fries_test_dir, fname) for fname in os.listdir(french_fries_test_dir)]

# Extract features for all images
features = []
for img_path in image_paths:
    try:
        features.append(extract_features(img_path, model))
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")

features = np.array(features)
print(f"Extracted features for {len(features)} images")

Extracted features for 1000 images


In [2]:
from sklearn.cluster import KMeans

# Number of clusters (choose based on expected subcategories, e.g., 3 types of pizza)
num_clusters = 6

# Perform K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(features)

# Get cluster labels for each image
cluster_labels = kmeans.labels_

# Print the cluster label for each image
for img_path, label in zip(image_paths, cluster_labels):
    print(f"{img_path} assigned to cluster {label}")

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


../data/food11/train/french_fries\1008163.jpg assigned to cluster 4
../data/food11/train/french_fries\1008945.jpg assigned to cluster 3
../data/food11/train/french_fries\1009455.jpg assigned to cluster 4
../data/food11/train/french_fries\1020588.jpg assigned to cluster 1
../data/food11/train/french_fries\1026314.jpg assigned to cluster 1
../data/food11/train/french_fries\1033213.jpg assigned to cluster 3
../data/food11/train/french_fries\1035421.jpg assigned to cluster 0
../data/food11/train/french_fries\1049122.jpg assigned to cluster 2
../data/food11/train/french_fries\1060646.jpg assigned to cluster 5
../data/food11/train/french_fries\1062594.jpg assigned to cluster 2
../data/food11/train/french_fries\1064632.jpg assigned to cluster 2
../data/food11/train/french_fries\1071481.jpg assigned to cluster 2
../data/food11/train/french_fries\1074446.jpg assigned to cluster 2
../data/food11/train/french_fries\1078563.jpg assigned to cluster 5
../data/food11/train/french_fries\1080215.jpg as

In [3]:
import shutil

# Create directories for each cluster (subcategories)
for i in range(num_clusters):
    train_subdir = os.path.join(french_fries_train_dir, f"cluster_{i}")
    test_subdir = os.path.join(french_fries_test_dir, f"cluster_{i}")
    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(test_subdir, exist_ok=True)

# Move images to subdirectories based on cluster labels
for img_path, label in zip(image_paths, cluster_labels):
    if 'train' in img_path:
        shutil.move(img_path, os.path.join(french_fries_train_dir, f"cluster_{label}", os.path.basename(img_path)))
    elif 'test' in img_path:
        shutil.move(img_path, os.path.join(french_fries_test_dir, f"cluster_{label}", os.path.basename(img_path)))

print("Images moved to their respective cluster directories.")

Images moved to their respective cluster directories.


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# Fine-grained apple_pie classification
french_fries_train_gen = ImageDataGenerator(rescale=1./255)
french_fries_test_gen = ImageDataGenerator(rescale=1./255)

french_fries_train_generator = french_fries_train_gen.flow_from_directory(
    french_fries_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

french_fries_test_generator = french_fries_test_gen.flow_from_directory(
    french_fries_test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Use the same base model (MobileNet)
base_model_french_fries = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model_french_fries.layers:
    layer.trainable = False

# Add custom layers for apple_pie classification
x_french_fries = base_model_french_fries.output
x_french_fries = GlobalAveragePooling2D()(x_french_fries)
x_french_fries = Dense(512, activation='relu')(x_french_fries)
french_fries_predictions = Dense(num_clusters, activation='softmax')(x_french_fries)

# Create and compile the model
french_fries_model = Model(inputs=base_model_french_fries.input, outputs=french_fries_predictions)
french_fries_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the fine-grained apple_pie classification model
french_fries_model.fit(
    french_fries_train_generator,
    steps_per_epoch=french_fries_train_generator.samples // 32,
    validation_data=french_fries_test_generator,
    validation_steps=french_fries_test_generator.samples // 32,
    epochs=30
)

# Save the model
french_fries_model.save('fine_grained_french_fries_classifier.h5')

Found 900 images belonging to 6 classes.
Found 100 images belonging to 6 classes.
Epoch 1/30
28/28 [==============================] - 9s 302ms/step - loss: 1.4158 - accuracy: 0.4721 - val_loss: 0.8758 - val_accuracy: 0.7396
Epoch 2/30
28/28 [==============================] - 7s 265ms/step - loss: 0.6723 - accuracy: 0.8191 - val_loss: 0.6348 - val_accuracy: 0.7708
Epoch 3/30
28/28 [==============================] - 7s 256ms/step - loss: 0.4541 - accuracy: 0.9090 - val_loss: 0.5237 - val_accuracy: 0.8646
Epoch 4/30
28/28 [==============================] - 7s 257ms/step - loss: 0.3365 - accuracy: 0.9412 - val_loss: 0.4320 - val_accuracy: 0.9062
Epoch 5/30
28/28 [==============================] - 7s 258ms/step - loss: 0.2750 - accuracy: 0.9574 - val_loss: 0.3859 - val_accuracy: 0.8958
Epoch 6/30
28/28 [==============================] - 7s 259ms/step - loss: 0.2200 - accuracy: 0.9724 - val_loss: 0.3534 - val_accuracy: 0.8854
Epoch 7/30
28/28 [==============================] - 7s 252ms/step 